In [1]:
import pandas as pd
import numpy as np
import scipy as sp
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
DFRAME_PATH="/content/drive/MyDrive/Colab Notebooks/aggregated_df.csv"

In [6]:
df = pd.read_csv(DFRAME_PATH)

In [7]:
df.head(10)

,author,book,text,text_no_punkt,lemmas,tags,tokens
0,Аверченко,Averchenko_A-T-Averchenko-Sobranie-sochineniy-...,Мы за пять лет. Материалы [к биографии]\nКак б...,Мы за пять лет Материалы к биографии Как будто...,пять год материал биография кроваво-красный ра...,4_NUMR 3_NOUN 9_NOUN 9_NOUN 15_ADJF 6_NOUN 8_V...,Мы за пять лет . Материалы [ к биографии ] Как...
1,Аверченко,Averchenko_A-T-Averchenko-Sobranie-sochineniy-...,Сидел он за большим письменным столом перед де...,Сидел он за большим письменным столом перед де...,сидеть больший письменный стол деревянный доск...,5_VERB 7_ADJF 10_ADJF 6_NOUN 10_ADJF 6_NOUN 6_...,Сидел он за большим письменным столом перед де...
2,Аверченко,Averchenko_A-T-Averchenko-Sobranie-sochineniy-...,"— Да уж, — качал головой сдержанный Ре-ми. — Н...",Да уж качал головой сдержанный Ре-ми Нехорошо ...,качать голова сдержать ре-ми нехорошо нехорошо...,5_VERB 7_NOUN 10_PRTF 5_None 8_ADVB 8_ADVB 4_A...,"— Да уж , — качал головой сдержанный Ре-ми . —..."
3,Аверченко,Averchenko_A-T-Averchenko-Sobranie-sochineniy-...,"Поверит ли кто-нибудь, что нами за эти пять ле...",Поверит ли кто-нибудь что нами за эти пять лет...,поверить кто-нибудь пять год совместно м.г кор...,7_VERB 10_NPRO 4_NUMR 3_NOUN 9_ADVB 3_None 11_...,"Поверит ли кто-нибудь , что нами за эти пять л..."
4,Аверченко,Averchenko_A-T-Averchenko-Sobranie-sochineniy-...,"8\nАверченко А. Избранные рассказы. М., 1985. ...",8 Аверченко А Избранные рассказы М 1985 С 7 9 ...,8 избранный рассказ м 1985 7 9 ежегодник 156 1...,1_None 9_ADJF 8_NOUN 1_NOUN 4_None 1_None 1_No...,"8 Аверченко А. Избранные рассказы . М. , 1985 ..."
5,Аверченко,Averchenko_A-T-Averchenko-Sobranie-sochineniy-...,"— Помилуйте, такой праздник… Нужно, обязательн...",Помилуйте такой праздник Нужно обязательно поц...,помиловать праздник нужно обязательно поцелова...,9_VERB 8_NOUN 5_PRED 11_ADVB 12_INFN 7_NOUN 10...,"— Помилуйте , такой праздник… Нужно , обязател..."
6,Аверченко,Averchenko_A-T-Averchenko-Sobranie-sochineniy-...,— Разбудить его в половине шестнадцатого.\n* *...,Разбудить его в половине шестнадцатого Закончи...,разбудить половина шестнадцатый закончить всё ...,9_INFN 8_NOUN 13_ADJF 8_GRND 3_PRCL 6_NOUN 7_N...,— Разбудить его в половине шестнадцатого . * *...
7,Аверченко,Averchenko_A-T-Averchenko-Sobranie-sochineniy-...,"Он встал, оделся, побрился и вышел на улицу.\n...",Он встал оделся побрился и вышел на улицу На у...,встать одеться побриться выйти улица улица сто...,5_VERB 6_VERB 8_VERB 5_VERB 5_NOUN 5_NOUN 11_V...,"Он встал , оделся , побрился и вышел на улицу ..."
8,Аверченко,Averchenko_A-T-Averchenko-Sobranie-sochineniy-...,Мозаика\nI\n— Я несчастный человек — вот что!\...,Мозаика I Я несчастный человек вот что Что за ...,мозаика i несчастный человек вздор поверить ув...,7_NOUN 1_None 10_ADJF 7_NOUN 5_NOUN 6_VERB 6_V...,Мозаика I — Я несчастный человек — вот что ! —...
9,Аверченко,Averchenko_A-T-Averchenko-Sobranie-sochineniy-...,— Как записываешь?\n— В записной книжке. Хочеш...,Как записываешь В записной книжке Хочешь У мен...,записывать записной книжка хотеть минута откро...,11_VERB 8_ADJF 6_NOUN 6_VERB 6_NOUN 13_NOUN 6_...,— Как записываешь ? — В записной книжке . Хоче...


In [8]:
def get_document_vectorizer(frame, n_min=1, n=2, max_count=10000, column="lemmas"):
    texts_vector = frame[column]
    vectorizer = TfidfVectorizer(ngram_range=(n_min,n), max_features=max_count, norm='l2')
    vectorizer.fit(texts_vector)
    return vectorizer

def get_author_vectorizer(frame, n_min=1, n=2, max_count=10000, column="lemmas"):
    grouped_text = frame.groupby("author", as_index = False)[column].agg({column: ' '.join})
    vectorizer = TfidfVectorizer(ngram_range=(n_min,n), max_features=max_count, norm='l2')
    texts_vector = frame[column]
    vectorizer.fit(texts_vector)
    return vectorizer

In [9]:
def get_encoder(frame, column="author"):
    encoder = LabelEncoder()
    encoder.fit(frame[column])
    return encoder


Возьмем лемматизированные отрезки текста, векторизируем их и попробуем предсказать

In [10]:
label_encoder = get_encoder(frame=df)

In [11]:
y = label_encoder.transform(df["author"])
df_train, df_test, y_train, y_test = train_test_split(df, y, train_size=0.5)

In [12]:
vectorizer = get_author_vectorizer(df_train, max_count=40000)
x_train = vectorizer.transform(df_train["lemmas"])
x_test = vectorizer.transform(df_test["lemmas"])

## CATBOOST

In [13]:
!pip install catboost # делал в коллабе, поэтому устанавливал каждый раз

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 76.6 MB 1.1 MB/s 


In [14]:
from catboost import Pool, CatBoostClassifier

In [15]:
df = df[["lemmas"]]


In [16]:
df_train, df_test, y_train, y_test = train_test_split(df, y, train_size=0.5)

In [23]:
train_pool = Pool(
    df_train, 
    y_train, 
    text_features=["lemmas"],
    
   
)
valid_pool = Pool(
    df_test, 
    y_test,
    text_features=["lemmas"], 
   
)

catboost_params = {
    'iterations': 15000,
    'learning_rate': 0.01,
    'eval_metric': 'Accuracy',
    'task_type': 'GPU',
    'early_stopping_rounds': 2000,
    'use_best_model': True,
    'verbose': 1000
}
model = CatBoostClassifier(**catboost_params)

In [24]:
model.fit(train_pool, eval_set=valid_pool)

0:	learn: 0.7303789	test: 0.7557510	best: 0.7557510 (0)	total: 33.7ms	remaining: 8m 25s
1000:	learn: 0.9371899	test: 0.9622237	best: 0.9622237 (977)	total: 22.4s	remaining: 5m 13s
2000:	learn: 0.9498196	test: 0.9648173	best: 0.9649301 (1876)	total: 42.5s	remaining: 4m 35s
3000:	learn: 0.9592918	test: 0.9663960	best: 0.9666216 (2878)	total: 1m 7s	remaining: 4m 30s
4000:	learn: 0.9660577	test: 0.9679747	best: 0.9679747 (3875)	total: 1m 27s	remaining: 4m
5000:	learn: 0.9728236	test: 0.9695535	best: 0.9695535 (4943)	total: 1m 47s	remaining: 3m 34s
6000:	learn: 0.9781236	test: 0.9701173	best: 0.9702300 (5996)	total: 2m 7s	remaining: 3m 10s
7000:	learn: 0.9829725	test: 0.9702300	best: 0.9703428 (6240)	total: 2m 26s	remaining: 2m 47s
8000:	learn: 0.9860171	test: 0.9711322	best: 0.9712449 (7947)	total: 2m 46s	remaining: 2m 25s
9000:	learn: 0.9884980	test: 0.9716960	best: 0.9718088 (8689)	total: 3m 6s	remaining: 2m 4s
10000:	learn: 0.9909788	test: 0.9720343	best: 0.9720343 (9817)	total: 3m 26s	